In [1]:
import pandas as pd
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
import io
import PyPDF2

In [2]:
urls = ["/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2022-2023/Fall/S1", 
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2022-2023/Fall/T1", 
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2022-2023/Fall/T2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2022-2023/Spring/T3",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2022-2023/Spring/T4",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2022-2023/Spring/S2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2021-2022/Fall",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2021-2022/Fall/T1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2021-2022/Fall/T2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2021-2022/Spring",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2021-2022/Spring/T3",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2021-2022/Spring/T4",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2020-2021/S1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2020-2021/S2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2020-2021/T1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2020-2021/T2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2020-2021/T3",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2020-2021/T4",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2019-2020/S1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2019-2020/S2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2019-2020/T1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2019-2020/T2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2019-2020/T3",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2019-2020/T4",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2018-2019/Fall Semester/Electives/S1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2018-2019/Fall Semester/Electives/T1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2018-2019/Fall Semester/Electives/T2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2018-2019/Spring Semester/Electives/S2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2018-2019/Spring Semester/Electives/T3",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2018-2019/Spring Semester/Electives/T4",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2017-2018/Fall Semester/Electives/S1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2017-2018/Fall Semester/Electives/T1",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2017-2018/Fall Semester/Electives/T2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2017-2018/Spring Semester/Electives/S2",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2017-2018/Spring Semester/Electives/T3",
        "/sites/SYLLABI-MastersCourses/Documentos Partilhados/General/Mestrados/MSc Syllabi 2017-2018/Spring Semester/Electives/T4"
        ]


In [3]:
# Step 1: Setup and Authenticate with Office365-REST-Python-Client
site_url = 'https://novasbe365.sharepoint.com/sites/SYLLABI-MastersCourses'
username = 'xxx'
password = 'xxx'

# Use UserCredential for authentication
user_credentials = UserCredential(username, password)
ctx = ClientContext(site_url).with_credentials(user_credentials)

course_data = []

for folder_url in urls:
    # Step 2: Directly access the folder using its server-relative URL extracted from the provided link
    folder = ctx.web.get_folder_by_server_relative_url(folder_url)

    # Load the folder to access its properties and files
    ctx.load(folder)
    files = folder.files
    ctx.load(files)
    ctx.execute_query()

    # Step 3: Read Each PDF and Populate DataFrame
    #course_data = []
    for file in files:
        if file.properties["Name"].lower().endswith('.pdf') and int(file.properties["Length"]) > 0:
            # Stream the file content
            file_content = file.open_binary_stream()
            ctx.execute_query()

            # Read PDF content using PyPDF2
            pdf_reader = PyPDF2.PdfFileReader(io.BytesIO(file_content.value))
            text = ""
            for page in range(pdf_reader.numPages):
                text += pdf_reader.getPage(page).extractText()
    
            # Extract necessary information from the text
            name = file.properties["Name"]
            syllabus = text  # Using the entire text as the syllabus

            course_data.append({"name": name, "syllabus": syllabus})

# Populate the DataFrame
courses = pd.DataFrame(course_data)

print(courses)

c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\a

Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'31' b'0'
Superfluous whitespace found in object header b'58' b'0'
Superfluous whitespace found in object header b'61' b'0'
Superfluous whitespace found in object header b'64' b'0'
Superfluous whitespace found in object header b'30' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object header b'20' b'0'
Superfluous whitespace found in object header b'19' b'0'
Superfluous whitespace found in object header b'18' b'0'
Superfluous whitespace found in object header b'28' b'0'
Superfluous whitespace found in object header b'27' b'0'
Superfluous whitespace found in object header b'26' b'0'
Superfluous whitespace found in obj

c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None


Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'47' b'0'
Superfluous whitespace found in object header b'46' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object header b'20' b'0'
Superfluous whitespace found in object header b'19' b'0'
Superfluous whitespace found in object header b'18' b'0'
Superfluous whitespace found in object header b'28' b'0'
Superfluous whitespace found in object header b'27' b'0'
Superfluous whitespace found in object header b'26' b'0'
Superfluous whitespace found in object header b'36' b'0'
Superfluous whitespace found in object header b'35' b'0'
Superfluous whitespace found in object header b'34' b'0'
Superfluous whitespace found in obj

c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None


...

c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\a

Xref table not zero-indexed. ID numbers for objects will be corrected.


c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\a

Xref table not zero-indexed. ID numbers for objects will be corrected.


c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\anaconda3\envs\DataSquad\lib\site-packages\certifi\cacert.pem None
c:\Users\celin\a

In [12]:
courses.to_csv("syllabi.csv", index=False, sep=';')